In [2]:
!pip install geopy

# Importation des bibliothèques

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import os
from IPython.display import IFrame
import time



# Chargement de la page web contenant les 50 festivals les plus populaires

In [4]:
# URL de la page
url = "https://infoconcert.com/festival/les-plus-consultes.html"

# Envoyer une requête HTTP
response = requests.get(url)
if response.status_code == 200:
    print("Page chargée avec succès")
else:
    print(f"Erreur lors du chargement de la page : {response.status_code}")
    exit()


Page chargée avec succès


# Extraction des données des festivals à partir de la page

In [5]:
from bs4 import BeautifulSoup

# Parser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Trouver les sections correspondant aux festivals
festival_rows = soup.find_all("div", class_="row")

# Extraire les données
festivals = []
seen = set()  # Pour éviter les doublons
for row in festival_rows:
    try:
        # Classement
        ranking_tag = row.find("span", class_="top-position-number")
        ranking = ranking_tag.text.strip() if ranking_tag else None

        # Nom du festival
        name_tag = row.find("div", class_="top-line-name").find("a")
        name = name_tag.text.strip() if name_tag else None
        link = f"https://infoconcert.com{name_tag['href']}" if name_tag else None

        # Lieu
        location_tag = row.find("div", class_="top-coming-concerts")
        location = location_tag.text.strip() if location_tag else None

        # Vérifier que les données sont complètes et uniques
        if name and ranking and (ranking, name) not in seen:
            festivals.append({
                "Classement": ranking,
                "Nom": name,
                "Lieu": location,
                "Lien": link
            })
            seen.add((ranking, name))  # Marquer comme vu
    except Exception as e:
        print(f"Erreur lors de l'extraction d'une ligne : {e}")

# Convertir en DataFrame
df = pd.DataFrame(festivals)

# Supprimer les doublons au cas où
df = df.drop_duplicates()

# Afficher un aperçu des données
print("Aperçu des festivals extraits :")
print(df.head())


Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Erreur lors de l'extraction d'une ligne : 'NoneType' object has no attribute 'find'
Aperçu des festivals extraits :
  Classement                      Nom                           Lieu  \
0         01    FESTIVAL RETRO C TROP                 Tilloloy\n(80)   
1         02        FESTIVAL DE NIMES                    Nimes\n(30)   
2         03        MONTJOUX FESTIVAL         Thonon Les Bains\n(74)   
3         04    PRINTEMPS DE PEROUGES  Saint Maurice de Remens\n(01)   
4         05  FESTIVAL DE CARCASSONNE              Carcassonne\n(11)   

                                                Lien  
0  https://infoconcert.com/festival/festival-retr...  
1  ht

# Nettoyage de la colonne "Lieu"

In [6]:
# Nettoyer la colonne "Lieu"
df['Lieu'] = df['Lieu'].str.replace(r'\n\(.*?\)', '', regex=True).str.strip()

# Vérifier le résultat
print("Lieux nettoyés :")
print(df[['Classement', 'Nom', 'Lieu']].head())


Lieux nettoyés :
  Classement                      Nom                     Lieu
0         01    FESTIVAL RETRO C TROP                 Tilloloy
1         02        FESTIVAL DE NIMES                    Nimes
2         03        MONTJOUX FESTIVAL         Thonon Les Bains
3         04    PRINTEMPS DE PEROUGES  Saint Maurice de Remens
4         05  FESTIVAL DE CARCASSONNE              Carcassonne


# Ajout des coordonnées géographiques des festivals

In [13]:
# Initialiser le géocodeur avec un délai
geolocator = Nominatim(user_agent="festival_scraper", timeout=10)

def geocode_location(location):
    try:
        print(f"Geocoding : {location}")
        geo = geolocator.geocode(location)
        if geo:
            return geo.latitude, geo.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erreur de géocodage pour {location}: {e}")
        return None, None

# Ajouter un délai entre les requêtes
df[['Latitude', 'Longitude']] = df['Lieu'].apply(
    lambda x: pd.Series(geocode_location(x) if pd.notna(x) else (None, None))
)
time.sleep(1)  # Délai pour éviter d'être bloqué par l'API

# Filtrer les résultats valides
df = df.dropna(subset=['Latitude', 'Longitude'])
print("Festivals avec coordonnées géographiques :")
print(df[['Nom', 'Lieu', 'Latitude', 'Longitude']].head())


Geocoding : Tilloloy
Geocoding : Saint Maurice de Remens
Geocoding : Saint Etienne
Geocoding : Luxey
Geocoding : Albi
Geocoding : La Rochelle
Geocoding : Luxembourg
Geocoding : Saint Julien En Genevois
Geocoding : Clisson
Geocoding : Gignac
Geocoding : Carhaix Plouguer
Geocoding : Argeles Sur Mer
Geocoding : Le Barcares
Geocoding : Bobital
Geocoding : Bordeaux
Geocoding : Monaco
Geocoding : Marmande
Geocoding : Ruoms
Geocoding : Camaret Sur Mer
Geocoding : Montreux
Geocoding : Cognac
Geocoding : Nort sur Erdre
Geocoding : Marseille
Geocoding : Clermont Ferrand
Geocoding : Nantes
Geocoding : Charleville Mezieres
Geocoding : Enghien
Geocoding : Surgeres
Geocoding : Landerneau
Geocoding : Aussonne
Festivals avec coordonnées géographiques :
                      Nom                     Lieu   Latitude  Longitude
0   FESTIVAL RETRO C TROP                 Tilloloy  49.644546   2.749010
3   PRINTEMPS DE PEROUGES  Saint Maurice de Remens  45.958549   5.278103
5     PAROLES ET MUSIQUES         

# Sauvegarde des données dans un fichier CSV

In [17]:
df.to_csv("50_festivals_geolocalises.csv", index=False, encoding="utf-8")
print("Les données ont été sauvegardées dans '50_festivals_geolocalises.csv'.")

Les données ont été sauvegardées dans '50_festivals_geolocalises.csv'.
